In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langserve import RemoteRunnable
from langchain_core.runnables import RunnableParallel
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
pdf_path = "../assets/240626_수요액티브전술집_LBM.pdf"

In [6]:
import fitz  # PyMuPDF
import io
from PIL import Image
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os

def extract_images_from_pdf(pdf_path):
    """PDF에서 이미지를 추출합니다."""
    doc = fitz.open(pdf_path)
    images = []
    for page in doc:
        image_list = page.get_images()
        for img in image_list:
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image = Image.open(io.BytesIO(image_bytes))
            images.append(image)
    return images

def image_to_base64(image):
    """이미지를 base64 인코딩된 문자열로 변환합니다."""
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

def summarize_image(image, llm):
    """이미지를 요약합니다."""
    base64_image = image_to_base64(image)
    
    system_message = SystemMessage(content="너는 이미지를 분석하고 설명하는 AI 어시스턴트야. 주어진 이미지를 간단히 설명해줘.")
    human_message = HumanMessage(content=f"이 이미지를 간단히 설명해주세요: data:image/png;base64,{base64_image}")
    
    messages = [system_message, human_message]
    
    response = llm.invoke(messages)
    return response.content

def extract_and_summarize(pdf_path):
    """PDF에서 이미지를 추출하고 각 이미지를 요약합니다."""
    llm = ChatOllama(model="qwen2:latest")
    
    images = extract_images_from_pdf(pdf_path)
    for i, image in enumerate(images[:3]):
        summary = summarize_image(image, llm)
        print(f"이미지 {i+1} 요약:")
        print(summary)
        print("-" * 50)

extract_and_summarize(pdf_path)

이미지 1 요약:
The provided ASCII art depicts a simple geometric design, featuring intersecting lines and shapes. Here's a breakdown of the elements:

1. **Lines**: The artwork includes several horizontal and vertical lines that create a grid-like structure at the top and bottom halves.

2. **Intersections**: At various points where these lines meet, smaller crosses or X-shaped patterns are formed by additional short lines crossing over each other.

3. **Shapes**: There are also small enclosed shapes created by intersecting lines near the center of the art piece. These might be interpreted as simple geometric figures like squares or rectangles, depending on how closely you look at their construction.

4. **Symmetry**: The design exhibits a certain level of symmetry. While not perfectly symmetrical across all axes due to the nature of the overlapping elements, there are clear attempts to balance the layout and maintain visual harmony.

5. **Complexity**: The interplay between simplicity and 

In [5]:
import fitz  # PyMuPDF
import pandas as pd

def extract_tables_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_tables = []
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        tables = page.find_tables()
        
        for table in tables:
            df = pd.DataFrame(table.extract())
            all_tables.append({
                'page': page_num + 1,
                'dataframe': df
            })
    
    doc.close()
    return all_tables

def print_table_info(tables):
    for i, table_info in enumerate(tables):
        print(f"Table {i+1} (Page {table_info['page']}):")
        print(table_info['dataframe'])
        print("\n" + "="*50 + "\n")

# 테이블 추출
extracted_tables = extract_tables_from_pdf(pdf_path)

# 추출된 테이블 정보 출력
# print_table_info(extracted_tables)

# 필요한 경우 DataFrame을 다른 형식으로 저장
# 예: CSV로 저장
# for i, table_info in enumerate(extracted_tables):
#     table_info['dataframe'].to_csv(f"table_{i+1}_page_{table_info['page']}.csv", index=False)

In [6]:
extracted_tables[0]

{'page': 3,
 'dataframe':         0                                  1  \
 0          거대언어모델(LLM; Large Language Model)   
 1   핵심 기능                         언어 이해 및 생성   
 2  데이터 종류                                텍스트   
 3   출력 형태                       텍스트, 그림, 비디오   
 4     도메인               PC, 스마트폰 등 온라인/온디바이스   
 5   활용 분야            자연어, 동영상 생성 등 텍스트 기반 응용   
 6   주요 기업                  오픈AI, 앤트로픽, 업스테이지   
 
                                    2  
 0  거대행동모델(LBM; Large Behavior Model)  
 1                          행동 및 의사결정  
 2                         촉각, 시각, 청각  
 3                          움직임, 상호작용  
 4           휴머노이드 로봇, 자율주행차 등 물리적 공간  
 5                           로봇 행동 응용  
 6          앱트로닉, 피규어AI, 생츄어리AI, 에이로봇  }

In [7]:
table_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """
당신은 표 형식의 데이터를 읽고 이해하여 자연어로 설명하는 전문가입니다. 
다음은 CSV 형식의 데이터입니다:

{csv_data}

데이터의 내용을 자연어로 풀어서 요약해주세요.
명확하고 논리적인 구조로 정보를 전달해주세요. 
절대 데이터의 출처나 형식(예: CSV 파일), 표의 행과 열에 대한 언급은 하지 마세요.

# answer :
"""
    ),
])

qwen2 = ChatOllama(model="qwen2:latest", temperature=0)
table_summary_chain = ({"csv_data": RunnablePassthrough()}
                       | table_summary_prompt 
                       | qwen2 
                       | StrOutputParser())

table_summary_chain.invoke(extracted_tables[0]["dataframe"])

'이 데이터는 거대언어모델(LLM)과 거대행동모델(LBM)에 관한 정보를 담고 있습니다. \n\nLLM은 핵심 기능을 언어 이해 및 생성으로, 데이터 종류가 텍스트이며 출력 형태는 텍스트, 그림 또는 비디오입니다. 이 모델의 도메인은 PC와 스마트폰 등의 온라인 및 온디바이스에 적용되며, 활용 분야에는 자연어 처리, 동영상 생성 등 다양한 텍스트 기반 응용 프로그램이 포함됩니다. 주요 기업으로는 오픈AI, 앤트로픽, 업스테이지가 있습니다.\n\nLBM의 핵심 기능은 행동 및 의사결정이며, 촉각, 시각, 청각 등의 감각을 포함하고 있습니다. 이 모델의 도메인은 물리적 공간에 적용되며, 활용 분야는 휴머노이드 로봇, 자율주행차 등과 같은 로봇 행동 응용 프로그램입니다. 주요 기업으로는 앱트로닉, 피규어AI, 생츄어리AI, 에이로봇 등이 있습니다.\n\n따라서 이 데이터는 두 가지 유형의 거대 모델에 대한 정보를 제공하고 있으며, 각각의 모델이 어떤 기능을 가지고 있는지, 그들이 어떻게 사용되는지 그리고 주요 기업은 무엇인지에 대해 설명하고 있습니다.'